In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import re
from ipywidgets import IntProgress
from IPython.display import display
from tqdm.auto import tqdm
from ordered_set import OrderedSet
from scipy.stats import sem, t
from scipy import stats as stats 
from scipy import mean
from nltk.metrics.distance import jaro_similarity
from nltk.metrics.distance import jaro_winkler_similarity

Гиперпараметры:

In [1715]:
min_score_limit = 0.7 #отсечка алгоритма Джаро-Винклера
max_length = 11 #максимальный префикс Джаро-Винклера
prefix_bias = 0.25 #вес в пользу общего префикса

Статистические параметры:

In [1716]:
accuracy = 0.3 #точность
interval = 0.05 #допустимое отклонение
confidence = 0.8 #достоверность
stat_power = 0.8 #статистическая мощность

Функция прогресса:

In [1717]:
def log_progress(sequence, every=1):
    progress = IntProgress(min=0, max=len(sequence), value=0)
    display(progress)
    
    for index, record in enumerate(sequence):
        if index % every == 0:
            progress.value = index
        yield record 

Функция поиска оптимального соответствия:

In [1718]:
def match(pattern, options):
    best_match = ''
    min_score = min_score_limit
    
    for item in options:
        
        p_set = OrderedSet(pattern.split())
        i_set = OrderedSet(item.split())         
        jaro_winkler_score = jaro_winkler_similarity(p_set, i_set, p=prefix_bias, max_l=max_length)
            
        if  jaro_winkler_score > min_score:
            min_score = jaro_winkler_score
            best_match = item
    
    return [best_match, round(min_score, 2)] 

Добавление нового элемента в серии:

In [1719]:
def append_ser(S):
    S = S.append(pd.Series(''), ignore_index=True)
    return S

Статистическая мощность:

In [1720]:
def get_power(n, p1, p2, cl):
    
    alpha = 1 - cl
    qu = stats.norm.ppf(1 - alpha/2)
    diff = abs(p2 - p1)
    bp = (p1 + p2) / 2
    
    v1 = p1 * (1 - p1)
    v2 = p2 * (1 - p2)
    bv = bp * (1 - bp)
    
    power_part_one = stats.norm.cdf((n**0.5 * diff - qu * (2 * bv)**0.5)/(v1 + v2)**0.5)
    power_part_two = 1 - stats.norm.cdf((n**0.5 * diff + qu * (2 * bv)**0.5)/(v1 + v2)**0.5)
    power = power_part_one + power_part_two
    
    return(power)

Минимальный объем выборки:

In [1721]:
def get_sample_size(power, p1, p2, cl, max_n = 100000):
    n = 1
    while n <= max_n:
        tmp_power = get_power(n, p1, p2, cl)
        if tmp_power >= power:
            return n
        else:
            n = n + 1

Читаем очищенный каталог устройств в df_devices:

In [1722]:
df_dev = pd.read_csv('data/devices_clean.csv') 
df_dev = df_dev.astype({'ven_d': 'str', 'model_d': 'str', 'tac_d': 'str'})
df_dev.head(0)

,ven_d,model_d,tac_d


Читаем очищенный каталог ЯМ-TOP в 'df_market':

In [4]:
df_market = pd.read_csv('data/market_clean.csv')[['ven_m', 'model_m', 'class_m']]
df_market = df_market.astype({'ven_m': 'str', 'model_m': 'str', 'class_m': 'str'})
df_market.head(0)

,ven_m,model_m,class_m


Созданием пустые серии для соответствия, дистанции и tac номера:

In [1724]:
match_series, tac_series, score_series = (pd.Series(), pd.Series(), pd.Series())

Поиск соответствий (match) и заполнение match_series, tac_series, score_series:

In [1725]:
for item in log_progress(df_market['model_m'], every=5):
    
    (match_series, tac_series) = (append_ser(match_series), append_ser(tac_series)) 
    score_series = append_ser(score_series)
    digit_relations = True
    
    market_vendor = df_market[df_market['model_m']==item]['ven_m'].values[0]
    
    if df_dev[df_dev['ven_d']==market_vendor].empty is False:
        
        device_models = list(df_dev[df_dev['ven_d'] == market_vendor]['model_d'])
        match_model = match(item, device_models)[0]
        match_score = match(item, device_models)[1]
        
        if re.search(r'\d+', item) and re.search(r'\d+', match_model):
            digit_relations = bool(re.search(r'\d+', item)[0]==re.search(r'\d+', match_model)[0])  
        
        if  digit_relations is True and (match_model is not ''):
            
            match_tac = df_dev[df_dev['model_d']==match_model]['tac_d'].values[0]           
            
            match_series = match_series[:-1].append(pd.Series(match_model), ignore_index=True)
            tac_series = tac_series[:-1].append(pd.Series(match_tac), ignore_index=True) 
            score_series = score_series[:-1].append(pd.Series(match_score), ignore_index=True)

IntProgress(value=0, max=7294)

Добавляем серии в базовый фрейм:

In [1726]:
df_match = pd.DataFrame({'model_d': match_series, 'score': score_series, 'tac_d': tac_series})
df_market = pd.concat([df_market, df_match] ,axis=1)
df_market = df_market.apply(lambda x: x.str.strip() if isinstance(x, str) else x).replace('', np.nan)

Создаем новый фрейм без пропущенных записей:

In [1727]:
df_market_matched = df_market.dropna(how='any', axis=0)

Выводим случайный сэмпл:

In [1728]:
df_market_matched.sample(n=10)

,ven_m,model_m,class_m,model_d,score,tac_d
3181,xiaomi,mi a2 lite 4 android one,мобильные телефоны,mi a2 lite m1805e2c,0.90,86870903
3170,huawei,y7 2019,мобильные телефоны,y7 prime 2019 dub tl20,0.84,86764704
2906,fly,life compact 4g,мобильные телефоны,life compact,0.93,35558809
3009,samsung,galaxy tab s 10 5 sm t800,планшеты,galaxy tab a 10 5 sm t597p,0.89,35840909
7160,samsung,galaxy j3 2016 sm j320f ds,мобильные телефоны,galaxy j3 2018 sm j337t,0.82,35573909
3229,xiaomi,redmi 7 4,мобильные телефоны,redmi 7 m1810f6lt,0.87,86609304
2788,xiaomi,mi max 3 4,мобильные телефоны,mi max 3 m1804e4c,0.93,86871003
2761,huawei,p rt 2019 3,мобильные телефоны,p smart 2019 pot al00,0.71,86891204
2849,nokia,1,мобильные телефоны,1 ta 1079,0.82,35447909
5332,samsung,galaxy tab a 10 1 sm t580,планшеты,galaxy tab a 10 5 sm t597v 1,0.96,35840809


Выводим число распознанных моделей:

In [1729]:
print('Доля распознанных паттернов:',round(100*len(df_market_matched)/len(df_market)), '%')

Доля распознанных паттернов: 2 %


Объем выборки с распознанными сущностями:

In [1730]:
print('Объем выборки с распознанными паттернами:',len(df_market_matched))

Объем выборки с распознанными паттернами: 153


Необходимый объем выборки для распознавания:

In [1731]:
sample_size = get_sample_size(stat_power, accuracy, accuracy*(1+interval), confidence)
print('Min. объем выборки для распознавания:', sample_size)

Min. объем выборки для распознавания: 8523


Интервал метрики Джаро-Винклера:

In [1732]:
h = sem(df_market_matched['score']) * t.ppf((1 + confidence) / 2, len(df_market_matched['score']) - 1)
trust_interval = (round(mean(df_market_matched['score'])-h,2), round(mean(df_market_matched['score'])+h,2))
print('Интервал:', trust_interval)

Интервал: (0.86, 0.87)


Статистики Джаро-Винклера:

In [1733]:
print('Медиана:',df_market_matched['score'].median())
print('Стандарт:',round(df_market_matched['score'].std(),2))
print('Min-Max:', str(df_market_matched['score'].min())+'-'+str(df_market_matched['score'].max()))

Медиана: 0.87
Стандарт: 0.07
Min-Max: 0.71-1.03


Записываем таблицу соответствий:

In [1734]:
df_market_matched.to_csv('data/market_matched.csv', index=False)